## 🧱 Özel Ortam

---

Bu görevde, derste örnek olarak kullanılan **ızgara tabanlı navigasyon ortamının** birebir aynısını tasarlayıp uygulayacaksınız.

- Ajan ızgaranın bir köşesinde başlar.
- Amaç karşı köşeye ulaşmaktır.
- Izgarada ajanın kaçınması gereken engeller veya "delikler" bulunabilir.

---

### 🎯 Amaçlar

- 📐 **Ortamı Tanımlayın**: ajanın başlangıçtan hedefe doğru hareket ettiği bir ızgara dünyası oluşturun.
- ⚙️ **Ortam Dinamiklerini Uygulayın**: hareket ve ödül ataması kurallarını programlayın (örn. deliklere düşmek için ceza, hedefe ulaşmak için ödül).
- 👁️ **Gözlem ve Eylemleri Ayarlayın**:
    - **Gözlem alanı** → Ajanın algıladığı şeyler (örn. ızgaradaki pozisyonu)  
    - **Eylem alanı** → Ajanın yapabileceği şeyler (örn. yukarı, aşağı, sola, sağa hareket)
- 🖼️ **Render Metodu Ekleyin**: ortamı görselleştirmek için `.render()` fonksiyonu ekleyin — hata ayıklama ve ajanın davranışını anlama için faydalıdır.
- 🧩 **Özel Özellikler Ekleyin**: derste ele alınan engeller, çukurlar veya diğer özellikleri dahil ederek ortamınızı daha dinamik ve gerçekçi hale getirin.

---
Bu görev için ihtiyacımız olan tüm paketleri içe aktararak başlayalım:

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Dict, Tuple


import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

---

### 🧩 Bölüm 1: Özel Ortam Sınıfı Oluşturma

Bu bölümde, kısmen yazılmış bir sınıfı tamamlayarak özel ortamınızı uygulayacaksınız.  
Yapı, Gymnasium'un ortam oluşturma için standart formatını takip eder.

### 🛠️ Yapacaklarınız

- Bazı temel metotları önceden yazılmış bir şablon sağlanmıştır.  
- `# CODE HERE` yazdığını gördüğünüz her yerde, eksik mantığı doldurmanız gerekir.  
- Her adımda size rehberlik etmesi için satır içi ipuçları verilmiştir.

### 🎯 Hedefiniz

- Sınıfı gerçek bir Gymnasium ortamı gibi davranacak şekilde tamamlayın.  
- Şunları yapmalı:
  - Geçerli bir gözlem ve eylem alanı tanımlamalı  
  - Ajan hareketini ve geçişleri ele almalı  
  - Ödülleri döndürmeli ve `done` bayraklarını uygun şekilde güncellemeli  
  - Çalışan bir `.reset()` ve `.step()` metodu içermeli  
  - İsteğe bağlı olarak görselleştirme için basit bir `.render()` içermeli

🧠 Her metodu anlamak için zaman ayırın — özellikle durum geçişlerinin ve ödüllerin nasıl yönetildiğini. RL ortamları burada canlanır.

📚 Ortam yapısı ve en iyi uygulamalar hakkında ayrıntılı adımlar için resmi [Gymnasium özel ortam kılavuzuna](https://gymnasium.farama.org/introduction/create_custom_env/) başvurun.

In [2]:
class CustomGridEnv(gym.Env):
    metadata = {'render_modes': ['human']}

    def __init__(self):
        # self.size = # Size of the square grid
        # self.action_space = # Define the action space to have four possible actions: right, up, left, down
        # self.agent_position = # Initial position of the agent (top-left corner)
        # self.goal_position =  # Position of the target (bottom-right corner)
        # self.hole_position =  # Position of the hole (bottom middle)
        # self.action_to_direction = # Mapping from actions to grid movements
        # 1. Ortamın Boyutu ve Değişkenleri
        self.size = 3  # 3x3 bir ızgara
        
        # 2. Eylem Alanı (Action Space): 4 hareket (Sağ, Yukarı, Sol, Aşağı)
        self.action_space = spaces.Discrete(4)

        # 3. Pozisyonlar
        self.agent_position = np.array([0, 0], dtype=np.int32) # Başlangıç (Sol Üst)
        self.goal_position = np.array([2, 2], dtype=np.int32)  # Hedef (Sağ Alt)
        self.hole_position = np.array([2, 1], dtype=np.int32)  # Delik (Alt Orta)

        # 4. Hareket Haritası (Eylemleri koordinat değişimine çevirir)
        # [satır_değişimi, sütun_değişimi] formatında
        self.action_to_direction = {
            0: np.array([0, 1]),   # Sağ
            1: np.array([-1, 0]),  # Yukarı
            2: np.array([0, -1]),  # Sol
            3: np.array([1, 0]),   # Aşağı
        }

        # Define the observation space using dictionary spaces for agent and target locations
        self.observation_space = spaces.Dict({
            "agent": spaces.Box(low=0, high=self.size - 1, shape=(2,), dtype=np.int32),
            "target": spaces.Box(low=0, high=self.size - 1, shape=(2,), dtype=np.int32),
        })

    def reset(self, seed: Optional[int] = None, options: Optional[Dict] = None) -> Tuple[Dict, Dict]:
        # Reset the environment for a new episode
        super().reset(seed=seed)

        # self.agent_position = # Reset the agent's position to the starting position
        # Ajanı başlangıç noktasına geri koy
        self.agent_position = np.array([0, 0], dtype=np.int32)

        # Goal position remains constant
        self.goal_position = np.array([2, 2], dtype=np.int32)

        # Generate observation and info for the current state
        observation = self._get_obs()
        info = self._get_info()
        return observation, info

    def _get_obs(self):
        # Return current observation of agent and target positions
        return {
            "agent": self.agent_position.copy(),
            "target": self.goal_position.copy()
        }

    def _get_info(self):
        # Compute the Manhattan distance between the agent and the target
        distance = np.sum(np.abs(self.agent_position - self.goal_position))
        return {"distance": distance}

    def step(self, action):
        # Apply the action to change the agent_position
        # If agent fell in hole, give -10 reward, and set done=True
        # elif agent reached goal, give +10 reward, and set done=True
        # else, give -1 reward, and set done=False
        # 1. Yeni pozisyonu hesapla
        direction = self.action_to_direction[action]
        
        # Mevcut pozisyona yönü ekle
        new_position = self.agent_position + direction

        # 2. Izgara sınırlarını kontrol et (Duvara çarpma)
        # np.clip, değerlerin 0 ile (size-1) arasında kalmasını sağlar
        self.agent_position = np.clip(new_position, 0, self.size - 1)

        # 3. Ödül ve Bitiş Durumlarını Kontrol Et
        
        # Ajan deliğe düştü mü?
        if np.array_equal(self.agent_position, self.hole_position):
            reward = -10
            done = True
        
        # Ajan hedefe ulaştı mı?
        elif np.array_equal(self.agent_position, self.goal_position):
            reward = 10
            done = True
        
        # Boş bir kareye adım attı
        else:
            reward = -1  # Her adım için küçük ceza (en kısa yolu bulması için)
            done = False

        # Collect observation and info about the current state
        observation = self._get_obs()
        info = self._get_info()
        return observation, reward, done, False, info  # 'truncated' is always False

    def render(self, mode='human'):

        # Visual representation of the grid
        grid = np.full((self.size, self.size), fill_value=' ')

        # Get x and y indexes for agent_position, goal_position, and hole_position
        # Koordinatları al (Satır, Sütun)
        agent_x, agent_y = self.agent_position
        goal_x, goal_y = self.goal_position
        hole_x, hole_y = self.hole_position

        # Izgara üzerine karakterleri yerleştir
        grid[agent_x][agent_y] = 'A'  # Agent
        grid[goal_x][goal_y] = 'G'    # Goal
        grid[hole_x][hole_y] = 'H'    # Hole

        # Görselleştirme
        print(f"\nGrid World ({self.size}x{self.size})")

        grid[agent_x][agent_y] = 'A'
        grid[goal_x][goal_y] = 'G'
        grid[hole_x][hole_y] = 'H'

        # Print the grid with borders
        print("+---" * self.size + "+")
        for row in grid:
            print("|" + "|".join(f" {cell} " for cell in row) + "|")
            print("+---" * self.size + "+")

👇 Önceki bölümü doğru tamamladığınızı test etmek için aşağıdaki hücreyi çalıştırın. Doğru yaptıysanız, ajanın bir bölümün sonu olan hedefe veya deliğe ulaşana kadar ortamınızda rastgele hareket ettiğini göreceksiniz.

In [3]:
# Create an instance of the custom environment
env = CustomGridEnv()

# Reset the environment to its initial state and get the initial observation and info
obs, info = env.reset()
print("Initial Observation:", obs)  # Display the initial position of the agent and the target
print("Initial Info:", info)  # Display additional information such as the distance from the target

# Loop through a maximum of 100 steps
for _ in range(100):
    action = env.action_space.sample()  # Randomly sample an action from the action space
    obs, reward, done, info, _ = env.step(action)  # Apply the action and get the results
    env.render()  # Render the current state of the environment to visualize the agent's position

    # Print the current state, reward received, whether the episode is done, and any additional info
    print(f"State: {obs}, Reward: {reward}, Done: {done}, Info: {info}")

    # If the episode is finished (agent reached the goal or fell into a hole), exit the loop
    if done:
        break


Initial Observation: {'agent': array([0, 0], dtype=int32), 'target': array([2, 2], dtype=int32)}
Initial Info: {'distance': 4}

Grid World (3x3)
+---+---+---+
|   | A |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([0, 1]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False

Grid World (3x3)
+---+---+---+
|   |   |   |
+---+---+---+
|   | A |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([1, 1]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False

Grid World (3x3)
+---+---+---+
|   |   |   |
+---+---+---+
|   |   | A |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([1, 2]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False

Grid World (3x3)
+---+---+---+
|   |   |   |
+---+---+---+
|   | A |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([1, 1]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Do

---
## Bölüm 2: DQN Eğitimi 🤖

Bu bölümde, özel ortamınızı başlatacak ve Stable Baselines3 kütüphanesini kullanarak bir DQN ajanı ile etkileşim için hazırlayacaksınız. Temel görev, ortamınızın kütüphanenin gereksinimleriyle uyumlu olduğundan emin olmaktır, bu da onu uygun şekilde sarmalamayı içerir.

#### 📝 İzlenecek adımlar

1. 🧱 **Özel Ortamı Başlatın**: özel ortam sınıfınızın bir örneğini oluşturun.
2. 🔁 **SB3 Uyumluluğunu Sağlayın**: Stable Baselines3'ten `make_vec_env` fonksiyonunu kullanarak ortamınızı sarın, böylece kütüphanenin vektörleştirilmiş ortam gereksinimleriyle uyumlu hale getirin.
3. ⚙️ **DQN Ajanını Yapılandırın ve Eğitin**: DQN ajanını uygun hiperparametrelerle kurun ve ortamınızda eğitin.
4. 📊 **Eğitim İlerlemesini İzleyin**: Ajanın öğrenme ilerlemesini ve performansını zaman içinde gözlemlemek için günlükleme ve izleme uygulayın.
5. 💾 **Modeli kaydedin**: Eğitim sonrasında modelinizi kaydedin.

In [4]:
# Ortamı vektörleştirilmiş hale getiriyoruz (SB3 için gereklidir)
# DummyVecEnv, tek bir ortamı sarmalayarak vektör ortamı gibi davranmasını sağlar
env = DummyVecEnv([lambda: CustomGridEnv()])

# PPO modelini başlatıyoruz
# 'MultiInputPolicy': Gözlem alanımız sözlük (Dict) olduğu için bu politikayı kullanıyoruz.
model = PPO("MultiInputPolicy", env, verbose=1)

print("Eğitim başlıyor...")
# Modeli 10.000 adım boyunca eğitiyoruz
model.learn(total_timesteps=10000)
print("Eğitim tamamlandı!")

# Eğitilmiş ajanı kaydet
model.save("ppo_custom_grid_agent")

Using cpu device
Eğitim başlıyor...
-----------------------------
| time/              |      |
|    fps             | 3512 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2476        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014654569 |
|    clip_fraction        | 0.242       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | 0.0159      |
|    learning_rate        | 0.0003      |
|    loss                 | 24.1        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0331     |
|    value_loss           | 61.2        |
----------------------------------------

---
## 🎮 Bölüm 3: Eğitilmiş modelinizi kullanın

Eğitilmiş DQN modelinizi yükleyecek ve ortamda karar vermek için kullanacaksınız, başlangıç noktasından hedefe delikleri kaçınarak nasıl gittiğini gözlemleyeceksiniz. Bu sadece eğitimin etkinliğini doğrulamanıza izin vermeyecek, aynı zamanda ajanın karar verme sürecini görsel olarak yorumlamanıza da olanak sağlayacaktır.

### İzlenecek adımlar 📝
1. 💾 **Eğitilmiş Modeli Yükleyin**: Eğitim aşamasında kaydedilen DQN modelini alın.
2. 🔄 **Ortamı Sıfırlayın**: Navigasyon görevini sıfırdan başlatmak için ortamı başlatın.
3. 🧠 **Navigasyon Simülasyonunu Çalıştırın**: Modeli kullanarak ortamın durumlarına dayalı eylemleri tahmin edin ve ajanın attığı her adımı görselleştirin.
4. 👀 **Her Adımı Görselleştirin**: Ajanın pozisyonunu, hedefi ve herhangi bir engeli veya deliği gösteren ızgaranın basit bir görselleştirmesini uygulayın.
5. 📝 **Ajan Davranışını Analiz Edin**: Ajanın hedefe ulaşma yeteneğini gözlemleyin ve not edin ve delikleri ne kadar etkili bir şekilde kaçındığını görün.

Kodun bir kısmı zaten orada ve `# CODE HERE` yorumunu gördüğünüz her yerde kod doldurmanız gerekiyor. Başlamanız için bazı ipuçları bulacaksınız.

In [7]:
import time
import numpy as np

# 1. Modeli Yükle (Eğitim sırasında kaydettiğimiz isimle)
model = PPO.load("ppo_custom_grid_agent")

obs = env.reset()  # Ortamı sıfırla ve ilk gözlemi al

for _ in range(200):  # Maksimum 200 adım (veya bölüm bitene kadar)
    
    # ---------------------------------------------------------
    # YOUR CODE HERE - BÖLÜM 1: Tahmin ve Adım
    # ---------------------------------------------------------
    # Mevcut gözleme dayanarak bir sonraki eylemi tahmin et
    # deterministic=True, ajanın eğitimden öğrendiği en iyi yolu izlemesini sağlar
    action, _states = model.predict(obs, deterministic=True)
    
    # Seçilen eylemi ortamda uygula, yeni durumu ve ödülü al
    obs, rewards, done, info = env.step(action)
    # ---------------------------------------------------------

    # Ortamı görsel olarak temsil etmek için boşluklarla dolu 3x3 bir ızgara başlat
    grid = np.full((3, 3), fill_value=' ')

    # Mevcut gözlemi kullanarak ajan, hedef ve delik için koordinatları çıkar
    # Not: DummyVecEnv kullandığımız için veriler liste içindedir (batch_size=1), bu yüzden [0] kullanıyoruz.
    agent_x, agent_y = obs['agent'][0][0], obs['agent'][0][1]
    goal_x, goal_y = obs['target'][0][0], obs['target'][0][1]
    hole_x, hole_y = 2, 1  # Deliğin konumu ortamda sabittir

    # Izgarayı ajanın, hedefin ve deliğin konumlarıyla güncelle
    grid[agent_x][agent_y] = 'A'  # Ajanın konumunu 'A' ile işaretle
    grid[goal_x][goal_y] = 'G'  # Hedef konumunu 'G' ile işaretle
    grid[hole_x][hole_y] = 'H'  # Delik konumunu 'H' ile işaretle

    # Mevcut durumu, alınan ödülü, bölümün bitip bitmediğini ve ek bilgileri yazdır
    # rewards[0] ve done[0] kullanıyoruz çünkü vektör ortamı dizi döndürür
    print(f"State: {obs}, Reward: {rewards[0]}, Done: {done[0]}, Info: {info}")

    # Izgarayı konsolda görsel olarak yazdır
    print("+---" * 3 + "+")  # Izgaranın üst sınırı
    for row in grid:
        print("|" + "|".join(f" {cell} " for cell in row) + "|")  # Her satırı hücreler '|' ile ayrılmış şekilde yazdır
        print("+---" * 3 + "+")  # Her satırdan sonra ayırıcı sınır

    # Adımları gözle takip edebilmek için kısa bir bekleme süresi (Opsiyonel ama önerilir)
    time.sleep(1)

    # ---------------------------------------------------------
    # YOUR CODE HERE - BÖLÜM 2: Bitiş Kontrolü
    # ---------------------------------------------------------
    # Eğer bölüm bittiyse (hedefe ulaşıldı veya ajan deliğe düştü)
    if done[0]:
        print("Bölüm Sona Erdi.")
        if rewards[0] > 0:
            print("🏆 Tebrikler! Hedefe ulaşıldı.")
        else:
            print("☠️ Ajan deliğe düştü.")
        
        # Ortamı sıfırla (veya döngüyü kır)
        obs = env.reset()
        break # Genellikle tek bir gösterimden sonra çıkmak daha temizdir.
    # ---------------------------------------------------------

State: OrderedDict({'agent': array([[0, 1]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: -1.0, Done: False, Info: [{'distance': 3, 'TimeLimit.truncated': False}]
+---+---+---+
|   | A |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: OrderedDict({'agent': array([[0, 2]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: -1.0, Done: False, Info: [{'distance': 2, 'TimeLimit.truncated': False}]
+---+---+---+
|   |   | A |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: OrderedDict({'agent': array([[1, 2]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: -1.0, Done: False, Info: [{'distance': 1, 'TimeLimit.truncated': False}]
+---+---+---+
|   |   |   |
+---+---+---+
|   |   | A |
+---+---+---+
|   | H | G |
+---+---+---+
State: OrderedDict({'agent': array([[0, 0]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: 10.0, Done: True, Info: [{'distance': 0, 'TimeLimit

🧠 Artık ajanınızın ortamda hareket ettiğini görmelisiniz.

Eğer ajan **belirli eylemleri tekrarlayarak takılıp kalırsa** veya hedefe ulaşamazsa, muhtemelen **yeterince uzun eğitilmediği** içindir.

Eğitim adım sayısını artırmayı deneyin ve modeli yeniden eğitin — daha uzun eğitim genellikle ajanın daha iyi bir politika öğrenmesine yardımcı olur.